<a href="https://colab.research.google.com/github/YoheiFukuhara/keras-for-beginner/blob/master/Keras06_Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[「【Keras入門(4)】Kerasの評価関数(Metrics)」](https://qiita.com/FukuharaYohei/items/f7df70b984a4c7a53d58)で解説しています。

In [0]:
from datetime import datetime
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, TensorBoard
from tensorflow.keras.metrics import TruePositives, TrueNegatives, FalsePositives, FalseNegatives, Precision, Recall

# データ数、バッチサイズの宣言

In [0]:
NUM_TRAIN = 128
BATCH_SIZE = 100

# 商と剰余の計算
tap_batch = divmod(NUM_TRAIN, BATCH_SIZE)

# 剰余があれば1加算
if tap_batch[1] == 0:
    batches = tap_batch[0]
else:
    batches = tap_batch[0] + 1

# テストデータの作成


## 説明変数
0から1までの乱数で128, 2の配列を作成

In [0]:
data = np.random.rand(NUM_TRAIN,2)

## 目的変数
説明変数の各行の2つの値を足した数

In [0]:
labels = (np.sum(data, axis=1) > 1.0) * 1
labels = labels.reshape(NUM_TRAIN,1)

# モデル定義

## 評価関数定義
以下のサイトを参考に実装

- https://blog.shikoan.com/keras-f1score/
- https://www.mathgram.xyz/entry/keras/backend
- https://zshitake.hatenablog.com/entry/2018/11/08/135335


In [0]:
# y_true が0/1なため、乗算すればOK。y_predは四捨五入して合計することでtpの値
def tp(y_true, y_pred):
    return K.sum(K.round(y_true * y_pred)) * batches

# y_trueが0/1で、y_predを加算したものを四捨五入し、0となったものをカウント(sum)する
def tn(y_true, y_pred):
    return K.sum(K.cast(K.equal(K.round(y_true + y_pred), 0), K.floatx())) * batches

def fp(y_true, y_pred):
    return K.sum(K.cast(K.equal(K.round(y_pred) - y_true, 1), K.floatx())) * batches

def fn(y_true, y_pred):
    return K.sum(K.cast(K.equal(y_true - K.round(y_pred), 1), K.floatx())) * batches

def custom_precision(y_true, y_pred):
    return tp(y_true, y_pred) / (tp(y_true, y_pred) + fp(y_true, y_pred))

def custom_recall(y_true, y_pred):
    return tp(y_true, y_pred) / (tp(y_true, y_pred) + fn(y_true, y_pred))


In [6]:
# Sequentialモデル使用(Sequentialモデルはレイヤを順に重ねたモデル)
model = Sequential()

# 全結合層(2層->4層)
model.add(Dense(32, input_dim=2, activation="tanh"))

# 結合層(4層->1層)：入力次元を省略すると自動的に前の層の出力次元数を引き継ぐ
model.add(Dense(1, activation="sigmoid"))

# 答え合わせのためにTensorFlow標準の評価関数も指定
model.compile(loss="binary_crossentropy", optimizer="sgd", metrics=[custom_precision, Precision(), custom_recall, Recall(), fp, FalsePositives(), tp, TruePositives(), tn, TrueNegatives(), fn, FalseNegatives()])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                96        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 129
Trainable params: 129
Non-trainable params: 0
_________________________________________________________________


In [0]:
# TensorBoardのログ保存先(タイムスタンプを付けておくと見るときに便利)
logdir = "log/run-{}/".format(datetime.utcnow().strftime("%Y%m%d%H%M%S"))

# CallBackの指定
li_cb = []
li_cb.append(TensorBoard(log_dir=logdir, write_graph=True, write_grads=True))

#訓練

In [8]:
model.fit(data, labels, epochs=300, callbacks=li_cb, batch_size=BATCH_SIZE)

Instructions for updating:
Use tf.cast instead.
Epoch 1/300
128/128 [==============================] - 1s 9ms/sample - loss: 0.7132 - custom_precision: nan - precision: 0.0000e+00 - custom_recall: 0.0000e+00 - recall: 0.0000e+00 - fp: 0.0000e+00 - false_positives: 0.0000e+00 - tp: 0.0000e+00 - true_positives: 0.0000e+00 - tn: 63.0000 - true_negatives: 63.0000 - fn: 65.0000 - false_negatives: 65.0000
Epoch 2/300
128/128 [==============================] - 0s 110us/sample - loss: 0.7120 - custom_precision: nan - precision: 0.0000e+00 - custom_recall: 0.0000e+00 - recall: 0.0000e+00 - fp: 0.0000e+00 - false_positives: 0.0000e+00 - tp: 0.0000e+00 - true_positives: 0.0000e+00 - tn: 63.0000 - true_negatives: 63.0000 - fn: 65.0000 - false_negatives: 65.0000
Epoch 3/300
128/128 [==============================] - 0s 136us/sample - loss: 0.7110 - custom_precision: nan - precision: 0.0000e+00 - custom_recall: 0.0000e+00 - recall: 0.0000e+00 - fp: 0.0000e+00 - false_positives: 0.0000e+00 - tp: 0.00

# ngrokをインストールしてTensorBoardを公開

In [9]:
#ngrok取得
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-06-13 12:56:04--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.73.9.93, 34.226.180.131, 35.173.6.94, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.73.9.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16648024 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  15.88M  15.1MB/s    in 1.1s    

2019-06-13 12:56:06 (15.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [16648024/16648024]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(logdir)
)
# Tunnel port 6006 (TensorBoard assumed running)
get_ipython().system_raw('./ngrok http 6006 &')

In [11]:
# ここで表示されたURLを開く
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://bbe6d36e.ngrok.io


In [12]:
# ライブラリバージョン確認
!pip freeze

absl-py==0.7.1
alabaster==0.7.12
albumentations==0.1.12
altair==3.1.0
astor==0.8.0
astropy==3.0.5
atari-py==0.1.15
atomicwrites==1.3.0
attrs==19.1.0
audioread==2.1.8
autograd==1.2
Babel==2.7.0
backcall==0.1.0
backports.tempfile==1.0
backports.weakref==1.0.post1
beautifulsoup4==4.6.3
bleach==3.1.0
blis==0.2.4
bokeh==1.0.4
boto==2.49.0
boto3==1.9.162
botocore==1.12.162
Bottleneck==1.2.1
branca==0.3.1
bs4==0.0.1
bz2file==0.98
cachetools==3.1.1
certifi==2019.3.9
cffi==1.12.3
chainer==5.4.0
chardet==3.0.4
cityhash==0.2.3.post9
Click==7.0
cloudpickle==0.6.1
cmake==3.12.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.1.3
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.14.6
cupy-cuda100==5.4.0
cvxopt==1.2.3
cvxpy==1.0.15
cycler==0.10.0
cymem==2.0.2
Cython==0.29.10
daft==0.0.4
dask==1.1.5
dataclasses==0.6
datascience==0.10.6
decorator==4.4.0
defusedxml==0.6.0
dill==0.2.9
distributed==1.25.3
Django==2.2.2
dlib==19.16.0
dm-sonnet==1.33
docopt==0.6.2
docutils==0.14